# Le data mining vs le machine learning

- Data Mining : recherche d’information et de patterns cachés dans une base de données isolée ou en croisant plusieurs bases. C’est l’être humain qui explore et prend des décisions.
- Le data mining s’appuie sur des techniques et algorithmes issus de la statistique et de la gestion de bases de données.
- Les données exploitées ont souvent une taille considérable.


# Quelques exemples

- Prestation de service : identification des paramètres de prédisposition à la résiliation (churn)
- Marketing: identification de segments distincts de consommateurs 
- Production : identification de paramètres induisant des défaillances
- Pharmaceutique: classification de médicaments selon le profil chimique


# Le machine learning

- Machine Learning : Techniques de calcul permettant aux ordinateurs d’apprendre à exécuter des tâches en s’inspirant de patterns perçus sur des données, avec peu d’intervention humaine. Les tâches se résument souvent à de la prédiction.

![image](./images/ml1.png)
    

![image](./images/trend-ml.png)

# Les outils

![image](./images/poll-ml.png)

# Les types de machine learning

- Les algorithmes de machine learning peuvent être rassemblés dans 3 groupes principaux :
   - L'apprentissage supervisé
   - Cible (variable dépendante qualitative ou quantitative)
   - Prédicteurs (variables indépendantes)
   - Régression, arbre de decision, foret aléatoire, kNN, regression logistique, SVM…

- L'apprentissage non supervisé
    - Pas de variable dépendante, pas de modélisation
    - Classer des individus dans différents groups ou trouver des associations
    - Apriori algorithm, K-means.

- L'apprentissage par renforcement
    - Apprentissage pour prendre des décisions spécifique afin d’aboutir à un but précis
    - Markov Decision Process

# L’analyse de données et le machine learning avec Python

Le package la plus adaptée pour faire du data mining et du machine learning est la package **scikit-learn**


Ce package propose des fonctions prédéfinies pour un grand nombre de méthodes

- La classification : SVM, plus proches voisins, random forest…
- Les régressions : linéaire, ridge, Lasso…
- Le clustering : k-means…
- L’analyse de données :  ACP, DA…

Dans le cadre de cette formation, l'objectf n'est pas de décrire la théorie des méthodes mais plutôt de comprendre l'utilisation de Python pour les appliquer.

# L’apprentissage supervisé avec Scikit-Learn
Les méthodes d’apprentissage supervisé sont les méthodes actuellement les plus
utilisées en data science. Il s’agit d’essayer de prédire une variable cible et d’utiliser
différentes méthodes pour arriver à cette fin.
Nous allons illustrer ces méthodes de traitement de données avec du code et des
cas pratiques.

### Les données et leur transformation
Ce jeu de données est décrit en détail au début du chapitre 4. Pour rappel, il est composé de 3333 individus et de 18 variables. Il est stocké dans un fichier csv, nommé telecom.csv, accessible dans le répertoire Data. On le récupère en utilisant Pandas :

In [1]:
import pandas as pd
import numpy as np

In [2]:
churn=pd.read_csv("./data/telecom.csv")

## Les données

Ce jeu de données n’a pas de données manquantes et nous allons devoir effectuer
quelques transformations pour l’adapter à nos traitements. Nous voyons par exemple qu’il est composé de trois colonnes object.

Nous pouvons afficher les statistiques descriptives pour les colonnes object :

In [3]:
churn.describe(include="object").transpose()

,count,unique,top,freq
State,3333,51,WV,106
Phone,3333,3333,339-5659,1
Int'l Plan,3333,2,no,3010
VMail Plan,3333,2,no,2411
Churn?,3333,2,False.,2850


## Transformation des données

On voit que les données sont toutes binaires. 

Pour les variables binaires, il nous suffit de les recoder avec Scikit-Learn pour obtenir des données exploitables. Par
ailleurs, il existe une autre variable qualitative dans notre jeu de données, Area Code,
qui est numérique mais avec trois modalités :

In [4]:
churn["Area Code"].value_counts()

415    1655
510     840
408     838
Name: Area Code, dtype: int64

## La préparation des données

Nous allons utiliser le processus de traitement classique pour transformer nos
données avec Scikit-Learn. Dans ce cas, nous n’avons pas de données manquantes,
nous travaillons donc sur la transformation des variables qualitatives.

In [5]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder,Imputer
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer 

In [6]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder,Imputer
#from sklearn.impute import SimpleImputer

LabelEncoder va nous permettre de transformer les valeurs textuelles en entiers.
Nous pouvons utiliser pour chaque variable qualitative :

In [7]:
# création d'un objet
encoder=LabelEncoder()
# on applique les données à l'objet
churn["Churn?"]=encoder.fit_transform(churn["Churn?"])

In [8]:
churn["Churn?"].value_counts()

0    2850
1     483
Name: Churn?, dtype: int64

In [9]:
encoder.inverse_transform(churn["Churn?"])

array(['False.', 'False.', 'False.', ..., 'False.', 'False.', 'False.'],
      dtype=object)

In [10]:
def fonc_transfo(frame):
    """ Fonction permettant de transformer toutes les colonnes qualitatives en quanti
    et de stocker les objets dans un dictionnaie
    """
    dict_label_encode={}
    for col in frame.columns:
        if frame[col].dtype == object:
            dict_label_encode[col]=LabelEncoder()
            frame[col]=dict_label_encode[col].fit_transform(frame[col])
    
    return frame, dict_label_encode

In [11]:
churn, dico_label = fonc_transfo(churn)

In [13]:
churn.head()

,State,Account Length,Area Code,Phone,Int'l Plan,VMail Plan,VMail Message,Day Mins,Day Calls,Day Charge,...,Eve Calls,Eve Charge,Night Mins,Night Calls,Night Charge,Intl Mins,Intl Calls,Intl Charge,CustServ Calls,Churn?
0,16,128,415,1926,0,1,25,265.1,110,45.07,...,99,16.78,244.7,91,11.01,10.0,3,2.70,1,0
1,35,107,415,1575,0,1,26,161.6,123,27.47,...,103,16.62,254.4,103,11.45,13.7,3,3.70,1,0
2,31,137,415,1117,0,0,0,243.4,114,41.38,...,110,10.30,162.6,104,7.32,12.2,5,3.29,0,0
3,35,84,408,1707,1,0,0,299.4,71,50.90,...,88,5.26,196.9,89,8.86,6.6,7,1.78,2,0
4,36,75,415,110,1,0,0,166.7,113,28.34,...,122,12.61,186.9,121,8.41,10.1,3,2.73,3,0


## Prédire l’attrition des clients
Lorsqu’on veut prédire une variable binaire, on devra avoir une colonne du type
binaire. On préfère généralement un codage 0/1 afin de garder un type entier simple à gérer. 

Les variables explicatives x auront été préparées de manière intelligente afin de bien appliquer nos modèles.

On crée donc x et y :

In [14]:
y = churn["Churn?"]
x= churn.select_dtypes(np.number)

## Séparation des données

Pour la séparation, on utilise la fonction train_test_split() de Scikit-Learn.

Cette fonction permet de créer automatiquement autant de structures que nécessaire
à partir de nos données. 

Elle utilise une randomisation des individus et ensuite une séparation en fonction d’un paramètre du type test_size :

In [15]:
# on importe la fonction
from sklearn.model_selection import train_test_split
# dans ce cas on a :
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.2,random_state=0)

Dans certains cas, il peut arriver qu’il y ait une forte disparité de distribution des
modalités entre les proportions d’acceptation et de refus. On peut vouloir faire en
sorte que les répartitions des modalités de y soient égales dans les différents échantillons,
on pourra alors utiliser une stratification. On va utiliser une stratification en
prenant y comme base pour effectuer la stratification :

In [16]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3,stratify = y)
y.value_counts(normalize=True)

0    0.855086
1    0.144914
Name: Churn?, dtype: float64

Ainsi les deux échantillons _train et _test ont la même distribution

In [17]:
y_test.value_counts(normalize=True)

0    0.855
1    0.145
Name: Churn?, dtype: float64

In [18]:
y_train.value_counts(normalize=True)

0    0.855122
1    0.144878
Name: Churn?, dtype: float64

## Le choix et l’ajustement de l’algorithme

Tout au long de ce Notebook, nous allons essayer d'ajouter un nouveau modèle, il s'agit du modèle GBM
```python
from sklearn.ensemble import GradientBoostingClassifier
```

In [19]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB

Ensuite, on crée un objet à partir de la classe du modèle en lui fournissant les
hyperparamètres dont il a besoin :

In [20]:
modele_rf=RandomForestClassifier(n_estimators=100,)
modele_knn=KNeighborsClassifier(n_neighbors=5)
modele_nb=GaussianNB()
modele_gbm=GradientBoostingClassifier()

Dans ce cas, on prend les hyperparamètres par défaut.

On peut ensuite ajuster notre modèle en utilisant les données :

In [21]:
%time modele_rf.fit(x_train,y_train)
%time modele_knn.fit(x_train,y_train)
%time modele_nb.fit(x_train,y_train)
%time modele_gbm.fit(x_train,y_train)
pass

Wall time: 422 ms
Wall time: 13 ms
Wall time: 5.98 ms
Wall time: 129 ms


Une fois qu’on a estimé les paramètres du modèle, on va pouvoir extraire des
informations. De nouveaux attributs de chaque classe apparaissent, ils se terminent par le symbole underscore _ :

In [22]:
modele_rf.feature_importances_

array([0.0053081 , 0.00559181, 0.00129665, 0.00480875, 0.03894956,
       0.00625659, 0.01101723, 0.04470434, 0.00733399, 0.04343462,
       0.01594849, 0.00532705, 0.0147657 , 0.00663377, 0.00471147,
       0.00876722, 0.01077326, 0.01460163, 0.01279262, 0.03567387,
       0.70130328])

Ce qui va nous intéresse avant tout, c’est de prédire avec notre modèle. Pour cela nous allons utiliser la méthode .predict() :

In [23]:
%time y_predict_rf = modele_rf.predict(x_test)
%time y_predict_knn = modele_knn.predict(x_test)
%time y_predict_nb = modele_nb.predict(x_test)
%time y_predict_gbm = modele_gbm.predict(x_test)

Wall time: 28 ms
Wall time: 72.8 ms
Wall time: 14.1 ms
Wall time: 2.99 ms


On obtient ainsi une valeur prédite pour les éléments de notre échantillon de
validation.

## Les indicateurs pour valider un modèle
La partie validation d’un modèle d’apprentissage supervisé est extrêmement
importante. L’objectif d’un modèle d’apprentissage supervisé est de prédire une
valeur la plus proche possible de la réalité. Nous différencions trois types d’indices
en fonction du type de variable cible. Tous les indicateurs de qualité du modèle sont
stockés dans le module *metrics* de Scikit-Learn.

## Le pourcentage de bien classés
Il s’agit de l’indicateur le plus connu. On le nomme accuracy. Il est calculé à partir du rapport entre le nombre d’individus bien classés et le nombre total d’individus dans l’échantillon.

In [24]:
from sklearn.metrics import accuracy_score, recall_score

accuracy_modele_rf = accuracy_score(y_test,y_predict_rf)
accuracy_modele_knn = accuracy_score(y_test,y_predict_knn)
accuracy_modele_nb = accuracy_score(y_test,y_predict_nb)
accuracy_modele_gbm = accuracy_score(y_test,y_predict_gbm)

print("Pourcentage de bien classés pour le modèle RF : %.3f" %(accuracy_modele_rf))
print("Pourcentage de bien classés pour le modèle kNN :%.3f" %(accuracy_modele_knn))
print("Pourcentage de bien classés pour le modèle naiveB :%.3f" %(accuracy_modele_nb))
print("Pourcentage de bien classés pour le modèle GBM :%.3f" %(accuracy_modele_gbm))

Pourcentage de bien classés pour le modèle RF : 1.000
Pourcentage de bien classés pour le modèle kNN :0.856
Pourcentage de bien classés pour le modèle naiveB :1.000
Pourcentage de bien classés pour le modèle GBM :1.000


In [25]:
recall_modele_rf=recall_score(y_test,y_predict_rf)
recall_modele_knn=recall_score(y_test,y_predict_knn)
print("Rappel pour le modèle RF : %.3f" %(recall_modele_rf))
print("Rappel pour le modèle kNN :%.3f" %(recall_modele_knn))

Rappel pour le modèle RF : 1.000
Rappel pour le modèle kNN :0.117


## La matrice de confusion
Il s’agit d’un autre indicateur important pour juger de la qualité d’un modèle, il n’est pas défini par une seule valeur mais par une matrice dans laquelle on peut lire le croisement entre les valeurs observées et les valeurs prédites à partir du modèle. 

Pour calculer cette matrice, on pourra utiliser :

In [26]:
from sklearn.metrics import confusion_matrix
confusion_matrix_rf=confusion_matrix(y_test,y_predict_rf)
confusion_matrix_knn=confusion_matrix(y_test,y_predict_knn)
confusion_matrix_nb=confusion_matrix(y_test,y_predict_nb)
confusion_matrix_gbm=confusion_matrix(y_test,y_predict_gbm)

print("Matrice de confusion pour le modèle RF :",
confusion_matrix_rf, sep="\n")
print("Matrice de confusion pour le modèle kNN :",
confusion_matrix_knn, sep="\n")
print("Matrice de confusion pour le modèle NB :",
confusion_matrix_nb, sep="\n")
print("Matrice de confusion pour le modèle GBM :",
confusion_matrix_gbm, sep="\n")

Matrice de confusion pour le modèle RF :
[[855   0]
 [  0 145]]
Matrice de confusion pour le modèle kNN :
[[839  16]
 [128  17]]
Matrice de confusion pour le modèle NB :
[[855   0]
 [  0 145]]
Matrice de confusion pour le modèle GBM :
[[855   0]
 [  0 145]]


## Le rappel (recall), la précision et le f1-score

Scikit-Learn possède des fonctions pour chacun de ces indicateurs, mais il peut
être intéressant d’utiliser une autre fonction qui les affiche pour chaque classe :

tp / (tp + fn)

In [27]:
from sklearn.metrics import classification_report
print("Rapport pour le modèle RF :",
      classification_report(y_test,y_predict_rf) ,sep="\n")

Rapport pour le modèle RF :
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       855
           1       1.00      1.00      1.00       145

    accuracy                           1.00      1000
   macro avg       1.00      1.00      1.00      1000
weighted avg       1.00      1.00      1.00      1000



In [28]:
print("Rapport pour le modèle kNN :",
      classification_report(y_test,y_predict_knn) ,sep="\n")

Rapport pour le modèle kNN :
              precision    recall  f1-score   support

           0       0.87      0.98      0.92       855
           1       0.52      0.12      0.19       145

    accuracy                           0.86      1000
   macro avg       0.69      0.55      0.56      1000
weighted avg       0.82      0.86      0.82      1000



## L’aire sous la courbe ROC
La courbe ROC est un indicateur important mais on préfère souvent une valeur plutôt
qu’une courbe afin de comparer nos modèles. Pour cela, on utilise l’aire sous la courbe
ROC (AUC). Cette aire est calculée directement à partir de la courbe ROC. Ainsi, un
modèle aléatoire aura une AUC de 0.5 et un modèle parfait aura une AUC de 1.

In [29]:
from sklearn.metrics import roc_auc_score
auc_modele_rf=roc_auc_score(y_test, modele_rf.predict_proba(x_test)[:,1])
auc_modele_knn=roc_auc_score(y_test,modele_knn.predict_proba(x_test)[:,1])
auc_modele_nb=roc_auc_score(y_test,modele_nb.predict_proba(x_test)[:,1])
auc_modele_gbm=roc_auc_score(y_test,modele_gbm.predict_proba(x_test)[:,1])

print("Aire sous la courbe ROC pour le modèle RF :" ,auc_modele_rf)
print("Aire sous la courbe ROC pour le modèle kNN :" ,auc_modele_knn)
print("Aire sous la courbe ROC pour le modèle NB :" ,auc_modele_nb)
print("Aire sous la courbe ROC pour le modèle GBM :" ,auc_modele_gbm)

Aire sous la courbe ROC pour le modèle RF : 1.0
Aire sous la courbe ROC pour le modèle kNN : 0.6172897761645493
Aire sous la courbe ROC pour le modèle NB : 1.0
Aire sous la courbe ROC pour le modèle GBM : 1.0


## La validation croisée
Jusqu’ici nous avons utilisé des indicateurs basés sur une seule occurrence de test. Ceci veut dire qu’on ne teste notre modèle que sur un seul échantillon.

Une approche alternative souvent utilisée est la validation croisée. Celle-ci est en fait basée sur la répétition de l’estimation et de la validation sur des données différentes.

Pour obtenir ce cv-score, on utilise :

In [30]:
from sklearn.model_selection import cross_val_score

scores_rf = cross_val_score(modele_rf, x_train, y_train, cv=5, scoring='roc_auc')
scores_knn = cross_val_score(modele_knn, x_train, y_train, cv=5, scoring='roc_auc')

print("AUC pour RF : %.3f (+/- %.3f)"% (scores_rf.mean(), scores_rf.std() * 2))
print("AUC pour kNN : %.3f (+/- %.3f)"% (scores_knn.mean(),scores_knn.std() * 2))

AUC pour RF : 1.000 (+/- 0.000)
AUC pour kNN : 0.587 (+/- 0.105)


## L’ajustement des hyperparamètres d’un modèle

L’une des tâches du data scientist est de trouver le meilleur modèle possible. La
plupart des modèles de machine learning ont des hyperparamètres. Il s’agit de paramètres
du modèle qui sont définis en amont de l’ajustement.

Scikit-Learn propose une classe GridSearchCV permettant d’implémenter cette
recherche d’hyperparamètres :

In [31]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

On va donc devoir définir les hyperparamètres que l’on souhaite tester. Pour cela,
on utilisera un dictionnaire d’hyperparamètres, par exemple :

In [32]:
dico_param= {"max_depth":[3,5,7,10], "n_estimators":[10,50,100,1000]}

On va encore utiliser l’accuracy pour valider notre modèle. Finalement, nous allons
utiliser une validation croisée à cinq groupes pour valider les résultats.
Le nouvel objet est le suivant :

In [33]:
recherche_hyper = GridSearchCV(RandomForestClassifier(), 
                               dico_param, 
                               scoring="accuracy",cv=4)

Une fois qu’on a créé cet objet, on peut lui joindre les données afin d’estimer les
meilleurs paramètres du modèle.

Cette étape peut être très longue.

In [34]:
recherche_hyper.fit(x, y)

GridSearchCV(cv=4, error_score='raise-deprecating',
             estimator=RandomForestClassifier(bootstrap=True, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators='warn', n_jobs=None,
                                              oob_score=False,
                                              random_state=None, verbose=0,
                                              warm_start=False),
             iid

In [35]:
print(recherche_hyper.best_params_)

{'max_depth': 5, 'n_estimators': 100}


In [36]:
print(recherche_hyper.best_score_)

1.0


In [37]:
pd.DataFrame(recherche_hyper.cv_results_)[["params","mean_test_score"]]

,params,mean_test_score
0,"{'max_depth': 3, 'n_estimators': 10}",0.946595
1,"{'max_depth': 3, 'n_estimators': 50}",0.985599
2,"{'max_depth': 3, 'n_estimators': 100}",0.993099
3,"{'max_depth': 3, 'n_estimators': 1000}",0.996100
4,"{'max_depth': 5, 'n_estimators': 10}",0.988899
5,"{'max_depth': 5, 'n_estimators': 50}",0.999400
6,"{'max_depth': 5, 'n_estimators': 100}",1.000000
7,"{'max_depth': 5, 'n_estimators': 1000}",1.000000
8,"{'max_depth': 7, 'n_estimators': 10}",0.999700
9,"{'max_depth': 7, 'n_estimators': 50}",1.000000


## La construction d’un pipeline de traitement

Bien souvent vous allez être amené à enchaîner des traitements sur des données.
On peut bien sûr développer son code de manière à suivre les étapes une à une mais il est souvent plus intéressant de créer des suites de traitements automatisées avec Scikit-Learn. 

Ces suites de traitements sont appelées pipeline. Ils simplifieront votre code et permettront de passer en production simplement.

Ainsi, on va pouvoir faire une analyse en composantes principales suivies d’un
algorithme de plus proches voisins directement dans un pipeline :

In [38]:
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.decomposition import PCA

In [39]:
acp=PCA(n_components=8)
#knn=KNeighborsClassifier()
rf=RandomForestClassifier()

In [40]:
composantes = acp.fit_transform(x)
rf.fit(composantes,y)

C:\Users\s4d-asus-14\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [41]:
composantes_test = acp.transform(x)
rf.predict(composantes_test)

array([0, 0, 0, ..., 0, 0, 0])

In [42]:

pipe2=Pipeline(steps=[("acp",PCA()),("rf",RandomForestClassifier())])
pipe2.fit(x, y)

C:\Users\s4d-asus-14\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Pipeline(memory=None,
         steps=[('acp',
                 PCA(copy=True, iterated_power='auto', n_components=None,
                     random_state=None, svd_solver='auto', tol=0.0,
                     whiten=False)),
                ('rf',
                 RandomForestClassifier(bootstrap=True, class_weight=None,
                                        criterion='gini', max_depth=None,
                                        max_features='auto',
                                        max_leaf_nodes=None,
                                        min_impurity_decrease=0.0,
                                        min_impurity_split=None,
                                        min_samples_leaf=1, min_samples_split=2,
                                        min_weight_fraction_leaf=0.0,
                                        n_estimators=10, n_jobs=None,
                                        oob_score=False, random_state=None,
                                        verbose=0, w

In [43]:
pipe2.predict(x)

array([0, 0, 0, ..., 0, 0, 0])

On a ainsi enchaîné deux traitements. Si on cherche des sorties liées à chacune
des étapes, on pourra le faire simplement. Par exemple, si l’objectif est d’extraire la part de variances expliquées par les composantes de l’analyse en composantes principales, on fera :

In [45]:
pipe2.named_steps["acp"].explained_variance_ratio_

array([9.85955067e-01, 3.25154733e-03, 2.78121524e-03, 2.70527664e-03,
       1.91317470e-03, 1.68656685e-03, 4.30288237e-04, 4.22903707e-04,
       4.03619953e-04, 2.34597638e-04, 1.98350089e-04, 8.91196877e-06,
       6.41312160e-06, 1.84109579e-06, 1.34833141e-07, 7.39849334e-08,
       1.78037265e-08, 8.81869864e-12, 8.70751746e-12, 8.37255932e-12,
       8.10577314e-12])

## Trouver la meilleure combinaison d’hyperparamètres dans un pipeline

Essayons de trouver la meilleure combinaison d’hyperparamètres dans un pipeline.
Dans le cadre de cet exemple, nous utiliserons les SVM (support vector machines,
également appelés séparateurs à vaste marge ou machines à vecteurs de support).
Ce sont des méthodes assez complexes dans leur principe mais simples dans leur
mise en oeuvre.

Les hyperparamètres d’un modèle de SVM sont assez nombreux. Les plus importants
étant le noyau choisi (linéaire, polynomial, sigmoïd, RBF…), les paramètres de
ces noyaux (le degré pour le cas polynomiale, gamma…) et le C pour la marge floue.

Dans notre exemple, on utilisera la fonction make_pipeline qui est équivalente
à la classe précédente avec quelques simplifications :

In [46]:
from sklearn.pipeline import make_pipeline
from sklearn.svm import SVC
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV

# construction du pipeline basé sur deux approches
mon_pipe=make_pipeline(PCA(), SVC(probability=True))

# construction du dictionnaire des paramètres
# (attention utilisation de __)
param_grid = dict(pca__n_components=[5, 10, x_train.shape[1]],
                  svc__C= [1, 10, 100, 1000],
                  svc__kernel= ['sigmoid', 'rbf'],
                  svc__gamma= [0.001, 0.0001])

# on construit l’objet GridSearch et on estime les hyper-paramètres
# par validation croisée
grid_search_mon_pipe = GridSearchCV(mon_pipe, param_grid = param_grid, scoring = "roc_auc", cv = 4)

In [47]:
grid_search_mon_pipe.fit(x_train,y_train)
pass

In [48]:
# la meilleure combinaisons de paramètres est :
grid_search_mon_pipe.best_params_

{'pca__n_components': 21,
 'svc__C': 1,
 'svc__gamma': 0.0001,
 'svc__kernel': 'rbf'}

In [49]:
grid_search_mon_pipe.best_score_

0.6703928791444584

Les meilleurs hyperparamètres obtenus en utilisant l’aire sous la courbe ROC sont
la combinaison C = 10, gamma = 0.0001, un noyau RBF pour les SVM et dix composantes
pour notre analyse en composantes principales.

Dans ce code, on définit les hyperparamètres associés à une méthode du pipeline
avec un double underscore : __.

L’utilisation des pipelines de Scikit-Learn va devenir rapidement une étape cruciale
de vos développements en Python.

## Passer en production votre modèle d’apprentissage supervisé

### Persistance de modèle avec Scikit-Learn

Python possède plusieurs outils pour la persistance d’objets, c’est-à-dire pour stocker
des objets dans des fichiers. Les objets de Scikit-Learn sont aussi dans cette
situation. On utilise un format pickle qui aura l’extension .pkl.

Par exemple, si nous voulons sauvegarder le dernier pipeline de traitement, nous
allons utiliser :

In [51]:
from sklearn.externals import joblib
joblib.dump(grid_search_mon_pipe, './data/modele_grid_pipe.pkl')

['./data/modele_grid_pipe.pkl']

Une fois ce modèle stocké, on peut très bien le réutiliser dans un autre cadre. Si
nous créons un nouveau notebook, nous allons utiliser :



In [52]:
from sklearn.externals import joblib
grid_search_mon_pipe = joblib.load('./data/modele_grid_pipe.pkl')

On peut ensuite appliquer le modèle avec tous les paramètres qui ont été appris :


```python
grid_search_mon_pipe.predict(x_test)
```

L’utilisation d’un fichier Pickle dans un notebook est une technique assez simple et courante.